In [3]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
#载入数据集
mnist = input_data.read_data_sets('/Users/ruicheng/Documents/上海师范研究生/tensorflow学习/MNIST_data',one_hot=True)

Extracting /Users/ruicheng/Documents/上海师范研究生/tensorflow学习/MNIST_data/train-images-idx3-ubyte.gz
Extracting /Users/ruicheng/Documents/上海师范研究生/tensorflow学习/MNIST_data/train-labels-idx1-ubyte.gz
Extracting /Users/ruicheng/Documents/上海师范研究生/tensorflow学习/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting /Users/ruicheng/Documents/上海师范研究生/tensorflow学习/MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
#每个批次的大小
batch_size = 100
#计算一共有多少批次
n_batch = mnist.train.num_examples // batch_size

#初始化权重
def weight_variable(shape):
    return tf.Variable(tf.truncated_normal(shape,stddev=0.1))

#初始化偏置
def bias_vairable(shape):
    return tf.Variable(tf.constant(0.1, shape=shape))

#卷积层
def conv2d(x,W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')

#池化层
def max_pool_2x2(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

x = tf.placeholder(tf.float32,[None,784])
y = tf.placeholder(tf.float32,[None,10])
keep_prob = tf.placeholder(tf.float32)

#改变X的格式转为4D的向量[batch,in_height,in_width,in_channels]
#黑白图片，in_channels=1
x_image = tf.reshape(x,[-1,28,28,1])

W_conv1 = weight_variable([5,5,1,32]) # 5*5的采样窗口，32个卷积核从1个平面抽取特征
b_conv1 = bias_vairable([32]) #每个卷积核一个偏置值

# 28*28*1 的图片卷积之后变为28*28*32
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
# 池化之后变为 14*14*32
h_pool1 = max_pool_2x2(h_conv1)

# 第二次卷积之后变为 14*14*64
W_conv2 = weight_variable([5,5,32,64])
b_conv2 = bias_vairable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1,W_conv2) + b_conv2)
# 第二次池化之后变为 7*7*64
h_pool2 = max_pool_2x2(h_conv2)


# 第一个全连接层
W_fc1 = weight_variable([7*7*64,1024])
b_fc1 = bias_vairable([1024])
# 7*7*64的图像变成1维向量
h_pool2_flat = tf.reshape(h_pool2,[-1,7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# 第二个全连接层
W_fc2 = weight_variable([1024,10])
b_fc2 = bias_vairable([10])
logits = tf.matmul(h_fc1_drop,W_fc2) + b_fc2
prediction = tf.nn.softmax(logits)

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=logits))
train_step = tf.train.AdamOptimizer(0.001).minimize(loss)

correct_prediction = (tf.equal(tf.argmax(prediction,1), tf.argmax(y,1)))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(21):
        for batch in range(n_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train_step, feed_dict={x:batch_xs,y:batch_ys,keep_prob:0.7})
        acc = sess.run(accuracy, feed_dict={x:mnist.test.images, y:mnist.test.labels, keep_prob:1.0})
        print("Iter: " + str(epoch) + ", acc: " + str(acc))

Iter: 0, acc: 0.9839


KeyboardInterrupt: 

In [1]:
import tensorflow as tf
import cv2
import numpy as np
import os         
from random import shuffle 
from tqdm import tqdm      
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

### 文件处理

In [2]:
TRAIN_DIR = 'train'
TEST_DIR = 'test'
IMG_SIZE = 56
LR = 1e-3
MODEL_NAME = 'dogs-vs-cats-convnet'

def create_label(image_name):
    """ Create an one-hot encoded vector from image name """
    word_label = image_name.split('.')[-3]
    if word_label == 'cat':
        return np.array([1,0])
    elif word_label == 'dog':
        return np.array([0,1])

def create_train_data():
    TRAIN_DIR = '/Users/ruicheng/Documents/上海师范研究生/毕业论文/猫狗识别(深度学习)/train-02'
    training_data = []
    for img in tqdm(os.listdir(TRAIN_DIR)):
        path = os.path.join(TRAIN_DIR, img)
        img_data = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        img_data = cv2.resize(img_data, (IMG_SIZE, IMG_SIZE))
        training_data.append([np.array(img_data), create_label(img)])
    shuffle(training_data)
    np.save('/Users/ruicheng/Documents/上海师范研究生/毕业论文/猫狗识别(深度学习)/train_data.npy', training_data)
    return training_data

def create_test_data():
    TEST_DIR = '/Users/ruicheng/Documents/上海师范研究生/毕业论文/猫狗识别(深度学习)/test'
    testing_data = []
    for img in tqdm(os.listdir(TEST_DIR)):
        path = os.path.join(TEST_DIR,img)
        img_num = img.split('.')[0]
        img_data = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        img_data = cv2.resize(img_data, (IMG_SIZE, IMG_SIZE))
        testing_data.append([np.array(img_data), img_num])
        
    shuffle(testing_data)
    np.save('/Users/ruicheng/Documents/上海师范研究生/毕业论文/猫狗识别(深度学习)/test_data.npy', testing_data)
    return testing_data

### 构造训练集和测试集

In [3]:
train_data = create_train_data()
test_data = create_test_data()
x = np.array([i[0] for i in train_data]).reshape(-1, IMG_SIZE, IMG_SIZE, 1).astype(np.float32)
y = np.array([i[1] for i in train_data]).astype(np.float32)
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.1,random_state=123)

100%|██████████| 5000/5000 [00:06<00:00, 817.98it/s]


In [4]:
x_train.dtype

dtype('float32')

In [ ]:
#初始化权重
def weight_variable(shape):
    return tf.Variable(tf.truncated_normal(shape,stddev=0.1))

#初始化偏置
def bias_vairable(shape):
    return tf.Variable(tf.constant(0.1, shape=shape))

#卷积层
def conv2d(x,W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')

#池化层
def max_pool_2x2(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

x_image = tf.placeholder(tf.float32,[None,IMG_SIZE,IMG_SIZE,1])
y = tf.placeholder(tf.float32,[None,2])
keep_prob = tf.placeholder(tf.float32)

#改变X的格式转为4D的向量[batch,in_height,in_width,in_channels]
#黑白图片，in_channels=1
# x_image = tf.reshape(x,[-1,56,56,1])

W_conv1 = weight_variable([5,5,1,32]) # 5*5的采样窗口，32个卷积核从1个平面抽取特征
b_conv1 = bias_vairable([32]) #每个卷积核一个偏置值

# 56*56*1 的图片卷积之后变为56*56*32
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
# 池化之后变为 28*28*32
h_pool1 = max_pool_2x2(h_conv1)

# 第二次卷积之后变为 28*28*64
W_conv2 = weight_variable([5,5,32,64])
b_conv2 = bias_vairable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1,W_conv2) + b_conv2)
# 第二次池化之后变为 14*14*64
h_pool2 = max_pool_2x2(h_conv2)

W_conv3 = weight_variable([5,5,64,128])
b_conv3 = bias_variable([128])
h_conv3 = tf.nn.relu(conv2d(h_pool2,W_conv2) + b_conv3)
# 第三次池化之后变为 7*7*128
h_pool3 = max_pool_2x2(h_conv3)

# 第一个全连接层
W_fc1 = weight_variable([(IMG_SIZE // 8)*(IMG_SIZE // 8)*128,1024])
b_fc1 = bias_vairable([1024])
# 7*7*64的图像变成1维向量
h_pool2_flat = tf.reshape(h_pool3,[-1,(IMG_SIZE // 8)*(IMG_SIZE // 8)*128])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# 第二个全连接层
W_fc2 = weight_variable([1024,2])
b_fc2 = bias_vairable([2])
logits = tf.matmul(h_fc1_drop,W_fc2) + b_fc2
prediction = tf.nn.softmax(logits)

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=logits))
train_step = tf.train.AdamOptimizer(0.001).minimize(loss)

correct_prediction = (tf.equal(tf.argmax(prediction,1), tf.argmax(y,1)))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(500):
        sess.run(train_step, feed_dict={x_image:x_train,y:y_train,keep_prob:0.7})
        acc = sess.run(accuracy, feed_dict={x:x_test, y:y_test, keep_prob:1.0})
        print("Iter: " + str(epoch) + ", acc: " + str(acc))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

